people on the samge group are one the same side

In [871]:
import pandas as pd
import numpy as np

df = pd.read_csv('pre_processed.csv')


In [872]:
def fill_deck_and_side_from_cabin(df):
    # Define a mask to identify rows where 'Number' is NaN and 'Cabin' is not NaN
    mask = df['Number'].isna() & df['Cabin'].notna()
    
    # Use the mask to update only the filtered rows, converting types appropriately
    df.loc[mask, ['Deck', 'Number', 'Side']] = df.loc[mask, 'Cabin'].apply(
        lambda x: pd.Series({
            'Deck': x.split('/')[0],
            'Number': int(x.split('/')[1]),  # Explicit conversion to integer here
            'Side': x.split('/')[2]
        })
    )

    return df

In [873]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

homeplanets = ['Earth', 'Europa', 'Mars']

all_cabin_sides = ['P','S']

all_cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [874]:
def multiple_decks_in_group(df,row):
    return len(df[df.Group == row.Group].dropna(subset ='Deck').Deck.unique()) > 1
        

In [875]:
def fill_potential_decks(df):
    def func_potential_decks(row):
        if pd.isna(row.Cabin):
            if row.Bills == 0 and not pd.isna(row.HomePlanet):
                if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
                    return decks_by_planet_no_bills[row.HomePlanet]
            if not pd.isna(row.HomePlanet):
                return decks_by_planet[row.HomePlanet]
            else:
                return all_cabin_decks
    df['potential_decks'] = df.apply(func_potential_decks, axis = 1)
    return df

In [876]:
def fill_potential_sides(df):
    def func_potential_sides(row):
        if pd.isna(row.Cabin):
            if row.GroupSize > 1:
                group = df[df.Group == row.Group].dropna(subset = 'Side')
                if len(group) > 0:
                    return [group.iloc[0].Side]
            return ['P','S']
    df['potential_sides'] = df.apply(func_potential_sides,axis = 1)
    return df

In [877]:
df = fill_potential_decks(df)
df = fill_potential_sides(df)

# Filling missed Cabins


In [878]:
def dict_of_utilised_cabins(df):
    used_cabins = {}
    for deck in all_cabin_decks:
        cab = {}
        for side in all_cabin_sides:
            rooms = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = 'Number').Number.unique())
            rooms.sort()
            cab[side] = rooms
        used_cabins[deck] = cab
    return used_cabins
    

#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

for row in dataframe

if cabin.isna() and homeplanet is known,
 for potential decks in homeplanets remit ie europa check a, b , c in dataframe up to that point and over that point, see if any have a free space and if not then fill it with the one in the same group

In [879]:
df_to_check.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 51
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  51
Number                51
Side                  51
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12919
potential_sides    12919
dtype: int64

In [880]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())
    return df

In [881]:
df.to_csv('before_impute.csv',index = False)

In [882]:
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()

Finding groups that have more than 1 member that all share the same cabin and that dont have other nans in group

In [883]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                299
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                 299
Number               299
Side                 299
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
potential_sides    12671
dtype: int64

## empty room with only one comptabile


should modify so that if there are multiple but its the sole member of its group it should take it 


In [937]:
df2 = fill_potential_decks(df2)
df2 = fill_potential_sides(df2)

In [938]:
def rooms_to_fill(df):
    rooms = {}
    for deck in all_cabin_decks:
        for side in all_cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

                    
                    
                    
            

In [939]:


def empty_room_one_compatible(df):
    count = 0
    empty_cabins = rooms_to_fill(df) 
    for deck in empty_cabins.keys():
        for side in empty_cabins[deck].keys():
            for number in empty_cabins[deck][side]:
                print("count",count)
                count +=1
                indices_of_compatible_rows = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                    if side not in row.potential_sides:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    before = list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())
                    if len(before) > 0:
                        if max(before) < number:
                            if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                                indices_of_compatible_rows.append(index)
                if len(indices_of_compatible_rows) == 1:
                    df.loc[indices_of_compatible_rows[0], 'Cabin'] = deck + "/" + str(int(number)) + "/" + side

    df = fill_deck_and_side_from_cabin(df)
    return df
                    
                    
                        

In [940]:
df2 = empty_room_one_compatible(df2)



count 0
count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22


In [941]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 31
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  31
Number                31
Side                  31
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12939
potential_sides    12939
dtype: int64

## No free rooms left so shares

In [942]:
def no_free_rooms_so_shares_3(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print()
        print(index)
        options = []
        for deck in passenger.potential_decks:
            for side in passenger.potential_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number
                if len(top_room) > 0:
                    top_room_number_before = max(top_room)
                else:
                    continue

                smallest_room = after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number
                
                if len(smallest_room) > 0:
                    smallest_room_number_after = min(smallest_room)
                else:
                    continue
                if top_room_number_before + 1 < smallest_room_number_after:   
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)
        if len(options) == 0:
            print(len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))].dropna(subset = 'Cabin').Cabin.unique()))
            if len(df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks)) & (df.Side.isin(passenger.potential_sides)) ].dropna(subset = 'Cabin').Cabin.unique()) == 1:
                df.loc[index,'Cabin'] = df[(df.Group == passenger.Group) & (df.Deck.isin(passenger.potential_decks))& (df.Side.isin(passenger.potential_sides))].dropna(subset = 'Cabin').Cabin.iloc[0]
    df = fill_deck_and_side_from_cabin(df)

    
    return df
    


In [943]:
df2 = fill_potential_sides(df2)

In [944]:
df2 = no_free_rooms_so_shares_3(df2)


404
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

421
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

479
[['E', 'P', 19.0, 22.0]]

505
[['E', 'P', 19.0, 22.0]]

517
[['E', 'P', 19.0, 22.0]]

1466
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

1543
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

2442
[]
3

2970
[]
3

3529
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]

3530
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]

4569
[]
2

4751
[]
2

5016
[['G', 'P', 589.0, 591.0], ['G', 'S', 578.0, 580.0]]

5017
[['G', 'P', 589.0, 591.0], ['G', 'S', 578.0, 580.0]]

8450
[['D', 'P', 190.0, 192.0], ['E', 'P', 386.0, 388.0]]

8465
[['D', 'P', 190.0, 192.0], ['E', 'P', 386.0, 388.0]]

10081
[['F', 'P', 1488.0, 1490.0], ['G', 'P', 1156.0, 1158.0]]

10082
[['F', 'P', 1488.0, 1490.0], ['G', 'P', 1156.0, 1158.0]]

10290
[['C', 'S', 269.0, 271.0]]

10313
[['C', 'S', 269.0, 271.0], ['D', 'P', 234.0, 236.0]]

10394
[['D', 'P', 234.0, 236.0], ['F', 'S', 1423.0, 1425.0]]

10408
[['

In [945]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 31
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  31
Number                31
Side                  31
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12939
potential_sides    12939
dtype: int64

# in a group of one and only one option

In [946]:
def solo_group_and_one_option(df):
    empty_cabins = rooms_to_fill(df) 
    
    for index,passenger in df[(df.GroupSize == 1) & (df.Cabin.isna())].iterrows():
        print()
        print("index",index)
        options = []
        for deck in empty_cabins.keys():
            if deck not in passenger.potential_decks:
                continue
            
            for side in empty_cabins[deck]:
                if side not in passenger.potential_sides:
                    continue
                for number in empty_cabins[deck][side]:
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    top_room = before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number
                    if len(top_room) > 0:
                        top_room_number_before = max(top_room)
                    else:
                        top_room_number_before = -1
                    smallest_room = after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number
                    
                    if len(smallest_room) > 0:
                        smallest_room_number_after = min(smallest_room)
                    else:
                        smallest_room_number_after = 1000
                    if top_room_number_before < number:
                        if smallest_room_number_after > number:
                            options.append([deck,side,top_room_number_before,smallest_room_number_after])
                        else:
                            print("srna",smallest_room_number_after,number)
                        
                            
                    else:
                        print("trnb",top_room_number_before,number)   
                            
        print(options)
                            
        if len(options) == 1:
            if options[0][2] + 2 == options[0][3]:
                print()
                df.loc[index,'Cabin'] = options[0][0] + "/" + str(int(options[0][2]) + 1) + "/" + options[0][1]
                print("found",index,options)
            else:
                print()
                print("multigap")
                print(index,options)
    df = fill_deck_and_side_from_cabin(df)
    
    return df
                
                    
                    

In [947]:
df2 = solo_group_and_one_option(df2)


index 404
srna 3.0 94
srna 14.0 40
srna 14.0 270
srna 14.0 298
srna 17.0 20
srna 17.0 21
srna 17.0 150
srna 17.0 387
srna 18.0 528
srna 11.0 191
srna 11.0 235
srna 11.0 36
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 421
srna 4.0 94
srna 14.0 40
srna 14.0 270
srna 14.0 298
srna 17.0 20
srna 17.0 21
srna 17.0 150
srna 17.0 387
srna 20.0 528
srna 12.0 191
srna 12.0 235
srna 11.0 36
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 1466
trnb 37.0 13
srna 8.0 94
trnb 39.0 13
srna 41.0 270
srna 41.0 298
trnb 58.0 20
trnb 58.0 21
srna 59.0 150
srna 59.0 387
srna 75.0 528
srna 40.0 191
srna 40.0 235
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 1543
trnb 38.0 13
srna 9.0 94
trnb 39.0 13
srna 41.0 270
srna 41.0 298
trnb 61.0 20
trnb 61.0 21
srna 62.0 150
srna 62.0 387
srna 76.0 528
srna 43.0 191
srna 43.0 235
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 3529
srna 520.0 1489
srna 520.0 1544
srna 484.0 1424
srna 406.0 590
srna 406.0 1157
srna 405.0 579
srna

In [948]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 31
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  31
Number                31
Side                  31
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12939
potential_sides    12939
dtype: int64

In [949]:

df2.loc[4233,'Cabin'] = 'B/98/P'
df2.loc[4254,'Cabin'] = 'B/99/P'
df2.loc[6493,'Cabin'] = 'E/300/S'
df2.loc[6514,'Cabin'] = 'E/301/S'
df2.loc[12892,'Cabin'] = 'F/1785/S'
df2.loc[12893,'Cabin'] = 'F/1785/S'
df2.loc[9265,'Cabin'] = 'F/1267/S'
df2.loc[9267,'Cabin'] = 'F/1267/S'
df2.loc[8413,'Cabin'] = 'A/57/P'
df2.loc[1429,'Cabin'] = 'E/58/P'

df2 = fill_deck_and_side_from_cabin(df2)


# fuctions to help

In [865]:
def all_cabin_options_for_each_row(df):
    count = 0
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print("\nindex", index)
        print("passenger",passenger.PassengerId)
        print("GroupSize", passenger.GroupSize)
        options = []
        for deck in passenger.potential_decks:
            for side in passenger.potential_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) or pd.isna(smallest_room_number_after):
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)
        print(passenger.potential_sides)
        if len(options) == 0:
            count += 1
    print(count)


In [866]:
def all_rows_for_each_cabin_option(df):
    empty_cabins = rooms_to_fill(df) 
    for deck in empty_cabins.keys():
        for side in empty_cabins[deck].keys():
            for number in empty_cabins[deck][side]:
                indices_of_compatible_rows = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                    if side not in row.potential_sides:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    before = list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())
                    after = list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())
                    if len(before) > 0:
                        if max(before) < number:
                            if len(after) > 0:
                                if min(after) > number:
                                    indices_of_compatible_rows.append(index)
                print()
                print("cabin", deck + "/" + str(int(number)) + "/" + side)
                print(indices_of_compatible_rows)

In [954]:
all_cabin_options_for_each_row(df_to_check)


index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]
['P', 'S']

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]
['P', 'S']

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57.0, 59.0]]
['P']

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]
['P', 'S']

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]
['P', 'S']

index 2442
passenger 1709_03
GroupSize 7
[]
['S']

index 2970
passenger 2092_03
GroupSize 5
[]
['S']

index 3529
passenger 2513_01
GroupSize 1
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]
['P', 'S']

index 3530
passenger 2514_01
GroupSize 1
[['E', 'P', 149.0, 151.0], ['F'

In [951]:
for index,row in df2.iterrows():
    if df2.iloc[index].Cabin != df_to_check.iloc[index].Cabin:
        print(index,df2.iloc[index].Cabin,df_to_check.iloc[index].Cabin)

404 nan nan
421 nan nan
479 nan nan
505 nan nan
517 nan nan
1429 E/58/P nan
1466 nan nan
1543 nan nan
2442 nan nan
2970 nan nan
3529 nan nan
3530 nan nan
4569 nan nan
4751 nan nan
5016 nan nan
5017 nan nan
5857 C/133/P nan
6133 C/139/P nan
6535 C/148/P nan
6578 B/156/P nan
6952 A/44/P nan
8102 B/197/P nan
8413 A/57/P nan
8450 nan nan
8465 nan nan
8534 A/58/P nan
8594 C/190/P nan
8822 A/63/P nan
8994 C/197/P nan
9273 C/206/P nan
9295 B/219/P nan
10081 nan nan
10082 nan nan
10127 C/235/P nan
10290 nan nan
10313 nan nan
10394 nan nan
10408 nan nan
10411 nan nan
10434 nan nan
10440 nan nan
10558 C/280/S nan
10855 B/301/S nan
11129 nan nan
11148 nan nan
11598 C/312/S nan
11999 C/324/S nan
12174 nan nan
12241 B/339/S nan
12651 nan nan
12668 nan nan


In [953]:
df_to_check.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 51
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  51
Number                51
Side                  51
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12919
potential_sides    12919
dtype: int64

In [ ]:

df2.loc[4233,'Cabin'] = 'B/98/P'
df2.loc[4254,'Cabin'] = 'B/99/P'
df2.loc[6493,'Cabin'] = 'E/300/S'
df2.loc[6514,'Cabin'] = 'E/301/S'
df2.loc[12892,'Cabin'] = 'F/1785/S'
df2.loc[12893,'Cabin'] = 'F/1785/S'
df2.loc[9265,'Cabin'] = 'F/1267/S'
df2.loc[9267,'Cabin'] = 'F/1267/S'
df2.loc[8413,'Cabin'] = 'A/57/P'
df2.loc[1429,'Cabin'] = 'E/58/P'

In [868]:
all_rows_for_each_cabin_option(df2)


cabin B/13/P
[404, 421]

cabin F/519/P
[3529, 3530]

cabin F/1489/P
[10081, 10082]

cabin F/1544/P
[10434, 10440]

cabin F/1424/S
[10394, 10408, 10411, 10434]

cabin A/94/P
[12651, 12668]

cabin G/590/P
[5016, 5017]

cabin G/1157/P
[10081, 10082]

cabin G/579/S
[5016, 5017]

cabin G/1206/S
[10408, 10411]

cabin G/1212/S
[10434, 10440]

cabin C/13/S
[404, 421]

cabin C/40/S
[1466, 1543]

cabin C/270/S
[10290, 10313]

cabin C/298/S
[11129, 11148]

cabin E/20/P
[479, 505, 517]

cabin E/21/P
[479, 505, 517]

cabin E/58/P
[1429, 1466]

cabin E/150/P
[3529, 3530]

cabin E/387/P
[8450, 8465]

cabin E/528/S
[11129, 11148]

cabin D/191/P
[8413, 8450, 8465]

cabin D/235/P
[10313, 10394]

cabin D/36/S
[1466, 1543]


no cryosleeps in T

In [933]:
all_cabin_options_for_each_row(df2)


index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]
['P', 'S']

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]
['P', 'S']

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]
['P']

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57.0, 59.0]]
['P']

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]
['P', 'S']

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]
['P', 'S']

index 2442
passenger 1709_03
GroupSize 7
[]
['S']

index 2970
passenger 2092_03
GroupSize 5
[]
['S']

index 3529
passenger 2513_01
GroupSize 1
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]
['P', 'S']

index 3530
passenger 2514_01
GroupSize 1
[['E', 'P', 149.0, 151.0], ['F'

In [934]:
df2[df2.Cabin == 'A/57/P']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks,potential_sides
8410,6028_01,Europa,False,A/57/P,55 Cancri e,27.0,False,0.0,1801.0,0.0,...,1,A,57.0,P,Wasatz,Preeldy,5,3498.0,None,None
8411,6028_02,Europa,False,A/57/P,TRAPPIST-1e,32.0,True,0.0,792.0,0.0,...,2,A,57.0,P,Cherkab,Preeldy,5,2128.0,None,None
8412,6028_03,Europa,False,A/57/P,55 Cancri e,11.0,False,0.0,0.0,0.0,...,3,A,57.0,P,Nusakar,Preeldy,5,0.0,None,None
8414,6028_05,Europa,False,A/57/P,TRAPPIST-1e,28.0,False,0.0,6921.0,0.0,...,5,A,57.0,P,Shelik,Preeldy,5,6995.0,None,None


In [935]:

df2.loc[4233,'Cabin'] = 'B/98/P'
df2.loc[4254,'Cabin'] = 'B/99/P'
df2.loc[6493,'Cabin'] = 'E/300/S'
df2.loc[6514,'Cabin'] = 'E/301/S'
df2.loc[12892,'Cabin'] = 'F/1785/S'
df2.loc[12893,'Cabin'] = 'F/1785/S'
df2.loc[9265,'Cabin'] = 'F/1267/S' # maybe only one is from this room and the other is joined in the other room
df2.loc[9267,'Cabin'] = 'F/1267/S' # with this one 
df2.loc[8413,'Cabin'] = 'A/57/P'
df2.loc[1429,'Cabin'] = 'E/58/P'

df2 = fill_deck_and_side_from_cabin(df2)


In [870]:
df2[df2.Group == 6028]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks,potential_sides
8410,6028_01,Europa,False,A/57/P,55 Cancri e,27.0,False,0.0,1801.0,0.0,...,1,A,57.0,P,Wasatz,Preeldy,5,3498.0,None,None
8411,6028_02,Europa,False,A/57/P,TRAPPIST-1e,32.0,True,0.0,792.0,0.0,...,2,A,57.0,P,Cherkab,Preeldy,5,2128.0,None,None
8412,6028_03,Europa,False,A/57/P,55 Cancri e,11.0,False,0.0,0.0,0.0,...,3,A,57.0,P,Nusakar,Preeldy,5,0.0,None,None
8413,6028_04,Europa,False,NaN,TRAPPIST-1e,26.0,True,0.0,3638.0,0.0,...,4,NaN,NaN,NaN,Kumark,Preeldy,5,4681.0,"[A, B, C, D, E, T]",[P]
8414,6028_05,Europa,False,A/57/P,TRAPPIST-1e,28.0,False,0.0,6921.0,0.0,...,5,A,57.0,P,Shelik,Preeldy,5,6995.0,None,None
